# Stock purchase recommendations with Machine Learning - Model Training

In [1]:
import pandas as pd
import numpy as np
#import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook # progress bar
import fastparquet
import pickle

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

from dask.distributed import Client, progress
import dask_ml.model_selection as dcv

from sklearn.externals import joblib

In [2]:
pd.set_option('display.max_columns', 1500)

## 1) Load Training and Test Data

In [3]:
# load the training and test datae from feature engineering step:
X_train = fastparquet.ParquetFile('../data/processed/X_train.parq').to_pandas()
X_test = fastparquet.ParquetFile('../data/processed/X_test.parq').to_pandas()
y_train = pickle.load(open('../data/processed/y_train.pkl', 'rb'))
y_test = pickle.load(open('../data/processed/y_test.pkl', 'rb'))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7024, 693), (1756, 693), (7024,), (1756,))

In [4]:
# quick inspection
X_train.tail()

,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-19,AdjOpen_-18,AdjOpen_-17,AdjOpen_-16,AdjOpen_-15,AdjOpen_-14,AdjOpen_-13,AdjOpen_-12,AdjOpen_-11,AdjOpen_-10,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-19,AdjHigh_-18,AdjHigh_-17,AdjHigh_-16,AdjHigh_-15,AdjHigh_-14,AdjHigh_-13,AdjHigh_-12,AdjHigh_-11,AdjHigh_-10,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-19,AdjLow_-18,AdjLow_-17,AdjLow_-16,AdjLow_-15,AdjLow_-14,AdjLow_-13,AdjLow_-12,AdjLow_-11,AdjLow_-10,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-19,AdjClose_-18,AdjClose_-17,AdjClose_-16,AdjClose_-15,AdjClose_-14,AdjClose_-13,AdjClose_-12,AdjClose_-11,AdjClose_-10,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_-19,AdjCloseRSI14_-18,AdjCloseRSI14_-17,AdjCloseRSI14_-16,AdjCloseRSI14_-15,AdjCloseRSI14_-14,AdjCloseRSI14_-13,AdjCloseRSI14_-12,AdjCloseRSI14_-11,AdjCloseRSI14_-10,AdjCloseRSI14_-9,AdjCloseRSI14_-8,AdjCloseRSI14_-7,AdjCloseRSI14_-6,AdjCloseRSI14_-5,AdjCloseRSI14_-4,AdjCloseRSI14_-3,AdjCloseRSI14_-2,AdjCloseRSI14_-1,AdjCloseRSI14_0,RSI_above_80_-19,RSI_above_80_-18,RSI_above_80_-17,RSI_above_80_-16,RSI_above_80_-15,RSI_above_80_-14,RSI_above_80_-13,RSI_above_80_-12,RSI_above_80_-11,RSI_above_80_-10,RSI_above_80_-9,RSI_above_80_-8,RSI_above_80_-7,RSI_above_80_-6,RSI_above_80_-5,RSI_above_80_-4,RSI_above_80_-3,RSI_above_80_-2,RSI_above_80_-1,RSI_above_80_0,RSI_below_20_-19,RSI_below_20_-18,RSI_below_20_-17,RSI_below_20_-16,RSI_below_20_-15,RSI_below_20_-14,RSI_below_20_-13,RSI_below_20_-12,RSI_below_20_-11,RSI_below_20_-10,RSI_below_20_-9,RSI_below_20_-8,RSI_below_20_-7,RSI_below_20_-6,RSI_below_20_-5,RSI_below_20_-4,RSI_below_20_-3,RSI_below_20_-2,RSI_below_20_-1,RSI_below_20_0,AdjCloseSMA10_-19,AdjCloseSMA10_-18,AdjCloseSMA10_-17,AdjCloseSMA10_-16,AdjCloseSMA10_-15,AdjCloseSMA10_-14,AdjCloseSMA10_-13,AdjCloseSMA10_-12,AdjCloseSMA10_-11,AdjCloseSMA10_-10,AdjCloseSMA10_-9,AdjCloseSMA10_-8,AdjCloseSMA10_-7,AdjCloseSMA10_-6,AdjCloseSMA10_-5,AdjCloseSMA10_-4,AdjCloseSMA10_-3,AdjCloseSMA10_-2,AdjCloseSMA10_-1,AdjCloseSMA10_0,AdjCloseSMA10_to_AdjClose_ratio_-19,AdjCloseSMA10_to_AdjClose_ratio_-18,AdjCloseSMA10_to_AdjClose_ratio_-17,AdjCloseSMA10_to_AdjClose_ratio_-16,AdjCloseSMA10_to_AdjClose_ratio_-15,AdjCloseSMA10_to_AdjClose_ratio_-14,AdjCloseSMA10_to_AdjClose_ratio_-13,AdjCloseSMA10_to_AdjClose_ratio_-12,AdjCloseSMA10_to_AdjClose_ratio_-11,AdjCloseSMA10_to_AdjClose_ratio_-10,AdjCloseSMA10_to_AdjClose_ratio_-9,AdjCloseSMA10_to_AdjClose_ratio_-8,AdjCloseSMA10_to_AdjClose_ratio_-7,AdjCloseSMA10_to_AdjClose_ratio_-6,AdjCloseSMA10_to_AdjClose_ratio_-5,AdjCloseSMA10_to_AdjClose_ratio_-4,AdjCloseSMA10_to_AdjClose_ratio_-3,AdjCloseSMA10_to_AdjClose_ratio_-2,AdjCloseSMA10_to_AdjClose_ratio_-1,AdjCloseSMA10_to_AdjClose_ratio_0,AdjClose_above_AdjCloseSMA10_-19,AdjClose_above_AdjCloseSMA10_-18,AdjClose_above_AdjCloseSMA10_-17,AdjClose_above_AdjCloseSMA10_-16,AdjClose_above_AdjCloseSMA10_-15,AdjClose_above_AdjCloseSMA10_-14,AdjClose_above_AdjCloseSMA10_-13,AdjClose_above_AdjCloseSMA10_-12,AdjClose_above_AdjCloseSMA10_-11,AdjClose_above_AdjCloseSMA10_-10,AdjClose_above_AdjCloseSMA10_-9,AdjClose_above_AdjCloseSMA10_-8,AdjClose_above_AdjCloseSMA10_-7,AdjClose_above_AdjCloseSMA10_-6,AdjClose_above_AdjCloseSMA10_-5,AdjClose_above_AdjCloseSMA10_-4,AdjClose_above_AdjCloseSMA10_-3,AdjClose_above_AdjCloseSMA10_-2,AdjClose_above_AdjCloseSMA10_-1,AdjClose_above_AdjCloseSMA10_0,AdjClose_crossed_AdjCloseSMA10_-19,AdjClose_crossed_AdjCloseSMA10_-18,AdjClose_crossed_AdjCloseSMA10

In [5]:
X_test.head()

,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-19,AdjOpen_-18,AdjOpen_-17,AdjOpen_-16,AdjOpen_-15,AdjOpen_-14,AdjOpen_-13,AdjOpen_-12,AdjOpen_-11,AdjOpen_-10,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-19,AdjHigh_-18,AdjHigh_-17,AdjHigh_-16,AdjHigh_-15,AdjHigh_-14,AdjHigh_-13,AdjHigh_-12,AdjHigh_-11,AdjHigh_-10,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-19,AdjLow_-18,AdjLow_-17,AdjLow_-16,AdjLow_-15,AdjLow_-14,AdjLow_-13,AdjLow_-12,AdjLow_-11,AdjLow_-10,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-19,AdjClose_-18,AdjClose_-17,AdjClose_-16,AdjClose_-15,AdjClose_-14,AdjClose_-13,AdjClose_-12,AdjClose_-11,AdjClose_-10,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_-19,AdjCloseRSI14_-18,AdjCloseRSI14_-17,AdjCloseRSI14_-16,AdjCloseRSI14_-15,AdjCloseRSI14_-14,AdjCloseRSI14_-13,AdjCloseRSI14_-12,AdjCloseRSI14_-11,AdjCloseRSI14_-10,AdjCloseRSI14_-9,AdjCloseRSI14_-8,AdjCloseRSI14_-7,AdjCloseRSI14_-6,AdjCloseRSI14_-5,AdjCloseRSI14_-4,AdjCloseRSI14_-3,AdjCloseRSI14_-2,AdjCloseRSI14_-1,AdjCloseRSI14_0,RSI_above_80_-19,RSI_above_80_-18,RSI_above_80_-17,RSI_above_80_-16,RSI_above_80_-15,RSI_above_80_-14,RSI_above_80_-13,RSI_above_80_-12,RSI_above_80_-11,RSI_above_80_-10,RSI_above_80_-9,RSI_above_80_-8,RSI_above_80_-7,RSI_above_80_-6,RSI_above_80_-5,RSI_above_80_-4,RSI_above_80_-3,RSI_above_80_-2,RSI_above_80_-1,RSI_above_80_0,RSI_below_20_-19,RSI_below_20_-18,RSI_below_20_-17,RSI_below_20_-16,RSI_below_20_-15,RSI_below_20_-14,RSI_below_20_-13,RSI_below_20_-12,RSI_below_20_-11,RSI_below_20_-10,RSI_below_20_-9,RSI_below_20_-8,RSI_below_20_-7,RSI_below_20_-6,RSI_below_20_-5,RSI_below_20_-4,RSI_below_20_-3,RSI_below_20_-2,RSI_below_20_-1,RSI_below_20_0,AdjCloseSMA10_-19,AdjCloseSMA10_-18,AdjCloseSMA10_-17,AdjCloseSMA10_-16,AdjCloseSMA10_-15,AdjCloseSMA10_-14,AdjCloseSMA10_-13,AdjCloseSMA10_-12,AdjCloseSMA10_-11,AdjCloseSMA10_-10,AdjCloseSMA10_-9,AdjCloseSMA10_-8,AdjCloseSMA10_-7,AdjCloseSMA10_-6,AdjCloseSMA10_-5,AdjCloseSMA10_-4,AdjCloseSMA10_-3,AdjCloseSMA10_-2,AdjCloseSMA10_-1,AdjCloseSMA10_0,AdjCloseSMA10_to_AdjClose_ratio_-19,AdjCloseSMA10_to_AdjClose_ratio_-18,AdjCloseSMA10_to_AdjClose_ratio_-17,AdjCloseSMA10_to_AdjClose_ratio_-16,AdjCloseSMA10_to_AdjClose_ratio_-15,AdjCloseSMA10_to_AdjClose_ratio_-14,AdjCloseSMA10_to_AdjClose_ratio_-13,AdjCloseSMA10_to_AdjClose_ratio_-12,AdjCloseSMA10_to_AdjClose_ratio_-11,AdjCloseSMA10_to_AdjClose_ratio_-10,AdjCloseSMA10_to_AdjClose_ratio_-9,AdjCloseSMA10_to_AdjClose_ratio_-8,AdjCloseSMA10_to_AdjClose_ratio_-7,AdjCloseSMA10_to_AdjClose_ratio_-6,AdjCloseSMA10_to_AdjClose_ratio_-5,AdjCloseSMA10_to_AdjClose_ratio_-4,AdjCloseSMA10_to_AdjClose_ratio_-3,AdjCloseSMA10_to_AdjClose_ratio_-2,AdjCloseSMA10_to_AdjClose_ratio_-1,AdjCloseSMA10_to_AdjClose_ratio_0,AdjClose_above_AdjCloseSMA10_-19,AdjClose_above_AdjCloseSMA10_-18,AdjClose_above_AdjCloseSMA10_-17,AdjClose_above_AdjCloseSMA10_-16,AdjClose_above_AdjCloseSMA10_-15,AdjClose_above_AdjCloseSMA10_-14,AdjClose_above_AdjCloseSMA10_-13,AdjClose_above_AdjCloseSMA10_-12,AdjClose_above_AdjCloseSMA10_-11,AdjClose_above_AdjCloseSMA10_-10,AdjClose_above_AdjCloseSMA10_-9,AdjClose_above_AdjCloseSMA10_-8,AdjClose_above_AdjCloseSMA10_-7,AdjClose_above_AdjCloseSMA10_-6,AdjClose_above_AdjCloseSMA10_-5,AdjClose_above_AdjCloseSMA10_-4,AdjClose_above_AdjCloseSMA10_-3,AdjClose_above_AdjCloseSMA10_-2,AdjClose_above_AdjCloseSMA10_-1,AdjClose_above_AdjCloseSMA10_0,AdjClose_crossed_AdjCloseSMA10_-19,AdjClose_crossed_AdjCloseSMA10_-18,AdjClose_crossed_AdjCloseSMA10

In [6]:
y_train

Index
2013-11-13    False
2013-11-13    False
2013-11-13    False
2013-11-13     True
2013-11-13     True
2013-11-13    False
2013-11-13     True
2013-11-13    False
2013-11-14    False
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-14     True
2013-11-14    False
2013-11-15     True
2013-11-15    False
2013-11-15    False
2013-11-15    False
2013-11-15     True
2013-11-15     True
2013-11-15     True
2013-11-15    False
2013-11-18    False
2013-11-18    False
2013-11-18     True
2013-11-18     True
2013-11-18     True
2013-11-18    False
              ...  
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-05    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-08    False
2017-05-09    False
2017-05-09    False
2017-05-09    False
2017-05-09    False
2017-05-09    

## 2) Build Initial Model - RandomForestClassifier

Start the ML process with a simple out of the box RandomForestClassifier to get a base line and validate that the training data is functioning with sklearn and can generate some predictions.

In [7]:
# define a simple pipeline
pipeline = Pipeline([
    ('standardScaler', StandardScaler()),
    ('randomForest', RandomForestClassifier())
])

# inspect parameters
pipeline.get_params()

{'memory': None,
 'randomForest': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'randomForest__bootstrap': True,
 'randomForest__class_weight': None,
 'randomForest__criterion': 'gini',
 'randomForest__max_depth': None,
 'randomForest__max_features': 'auto',
 'randomForest__max_leaf_nodes': None,
 'randomForest__min_impurity_decrease': 0.0,
 'randomForest__min_impurity_split': None,
 'randomForest__min_samples_leaf': 1,
 'randomForest__min_samples_split': 2,
 'randomForest__min_weight_fraction_leaf': 0.0,
 'randomForest__n_estimators': 'warn',
 'randomForest__n_jobs': None,
 'randomForest__oob_score': F

In [8]:
%%timeit -r1 -n1

# fit the pipeline with all default parameters
pipeline.fit(X_train, y_train.reset_index().setup_for_profitable_trade)

C:\Users\florian.lindstaedt\AppData\Local\Continuum\anaconda3\envs\trader_env2\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\florian.lindstaedt\AppData\Local\Continuum\anaconda3\envs\trader_env2\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\florian.lindstaedt\AppData\Local\Continuum\anaconda3\envs\trader_env2\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1 loop, best of 1: 2.18 s per loop


In [9]:
# create prediction
y_pred_firstRF = pipeline.predict(X_test)

# save for backtesting in separate notebook
pickle.dump(y_pred_firstRF, open('../data/model_predictions/y_pred_firstRF.pkl', 'wb'))

C:\Users\florian.lindstaedt\AppData\Local\Continuum\anaconda3\envs\trader_env2\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


We now have a first prediction on the dataset - let's look into the performance of the default settings. To understand the result, we will look at a few different metrics: the accuracy, precision, recall, f1-score, and the confusion matrix.

It is important to note that our data is imbalanced (not the same number of days with trade signal as with no signal). Therefore, F scores are a better metric than ROC-AUC or accuracy. We can not easily fix the imbalance since we are looking at time series data and resampling might introduce lookahead bias.

The ultimate test of the quality of the prediction is to backtest the results (ie simulate financial performance based on the predictions). This will be done in the next workbook. For now let's inspect some of the basic metrics.

In [10]:
print(classification_report(y_test, y_pred_firstRF))

              precision    recall  f1-score   support

       False       0.64      0.81      0.72      1153
        True       0.26      0.12      0.17       603

   micro avg       0.58      0.58      0.58      1756
   macro avg       0.45      0.47      0.44      1756
weighted avg       0.51      0.58      0.53      1756



In [11]:
accuracy_score(y_test, y_pred_firstRF)

0.576879271070615

In [12]:
confusion_matrix(y_test, y_pred_firstRF, labels=[False, True])

array([[939, 214],
       [529,  74]], dtype=int64)

The confusion matrix indicates a large number of False Positives (bad because if we used those to trade, we would enter a trade that turns out not to be as profitable as desired) and False Negatives (bad because they mean missed opportunities for us to enter profitable trades).

The good news is that our out-of-the-box model was able to predict some True Positives but the results in terms of financial performance are most likely very bad (we saved off the prediction and will look at this later).

Even though the model seems to perform poorly, we might be able to learn something from it:

Let's look into the relative feature importances to see if anything stands out

In [13]:
def print_feature_importances(estimator):

    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print(str(f + 1) + " importance: " + str(importances[indices[f]]) + ". feature name: " + X_train.columns[indices[f]])

In [14]:
      print_feature_importances(pipeline.named_steps['randomForest'])

Feature ranking:
1 importance: 0.007503600762375867. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-14
2 importance: 0.00728257777202395. feature name: AdjCloseSMA50_chg_-12
3 importance: 0.006759803611065378. feature name: AdjCloseSMA50_chg_-19
4 importance: 0.00606914599062009. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-17
5 importance: 0.005965864840202175. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-16
6 importance: 0.005924810604282437. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-4
7 importance: 0.005748204169308406. feature name: AdjCloseSMA50_chg_-15
8 importance: 0.0056239864494624925. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-8
9 importance: 0.005173509775227882. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-2
10 importance: 0.005152900102246462. feature name: AdjCloseSMA200_-9
11 importance: 0.005146306865472338. feature name: AdjCloseSMA50_-18
12 importance: 0.005101738043150556. feature name: AdjCloseSMA200_chg_-17
13 impor

105 importance: 0.002809615687173652. feature name: AdjCloseSMA50_-12
106 importance: 0.002797746049267326. feature name: AdjCloseSMA50_chg_-5
107 importance: 0.002762136005047145. feature name: AdjHigh_chg_-18
108 importance: 0.0027512268241640667. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-4
109 importance: 0.002747712813325355. feature name: AdjCloseSMA10_to_AdjClose_ratio_-14
110 importance: 0.0027476049843828012. feature name: AdjCloseSMA200_chg_-19
111 importance: 0.002736232430906562. feature name: AdjCloseSMA50_-17
112 importance: 0.0027240443462283282. feature name: AdjCloseSMA10_chg_-11
113 importance: 0.0027233126083755477. feature name: AdjCloseRSI14_-18
114 importance: 0.0027165491966218095. feature name: AdjCloseSMA10_to_AdjClose_ratio_-1
115 importance: 0.0027055462685955174. feature name: AdjCloseSMA200_chg_-13
116 importance: 0.0027048337846881333. feature name: AdjClose_-4
117 importance: 0.002697527716285882. feature name: AdjCloseSMA50_-5
118 importance: 0.

218 importance: 0.002002389027579946. feature name: AdjCloseRSI14_-13
219 importance: 0.0020019528722211503. feature name: AdjClose_chg_-2
220 importance: 0.0019905667264656195. feature name: AdjHigh_chg_-12
221 importance: 0.0019872685226783988. feature name: AdjHigh_chg_-16
222 importance: 0.0019803726112112984. feature name: AdjHigh_chg_-15
223 importance: 0.001972054305247126. feature name: AdjLow_chg_-19
224 importance: 0.0019623828179847766. feature name: AdjCloseSMA200_chg_-6
225 importance: 0.001960273661563031. feature name: AdjVolume_-11
226 importance: 0.001955672670476408. feature name: AdjLow_chg_-15
227 importance: 0.0019528083135676257. feature name: AdjLow_chg_-17
228 importance: 0.0019501571836897536. feature name: AdjCloseRSI14_-16
229 importance: 0.0019464855910394957. feature name: AdjVolume_-14
230 importance: 0.0019428029696303552. feature name: AdjCloseSMA50_-13
231 importance: 0.0019334840393606112. feature name: AdjLow_chg_-18
232 importance: 0.0019246117600566

332 importance: 0.0015217236579924262. feature name: AdjOpen_-19
333 importance: 0.0015167842681320927. feature name: AdjClose_chg_0
334 importance: 0.0015154224846364883. feature name: AdjCloseSMA200_to_AdjClose_ratio_-17
335 importance: 0.001515246604021658. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-19
336 importance: 0.0015144164093160684. feature name: AdjOpen_chg_-2
337 importance: 0.0015134763141982097. feature name: AdjLow_chg_-11
338 importance: 0.0015125498485465772. feature name: AdjVolume_chg_-8
339 importance: 0.0015116649692713483. feature name: AdjCloseSMA50_to_AdjClose_ratio_-11
340 importance: 0.0014920887437206787. feature name: AdjClose_-12
341 importance: 0.001489270805557875. feature name: AdjClose_chg_-8
342 importance: 0.0014879763950151458. feature name: AdjCloseRSI14_-1
343 importance: 0.001486832370074645. feature name: AdjCloseSMA50_to_AdjClose_ratio_-6
344 importance: 0.0014852373589875208. feature name: AdjOpen_chg_-12
345 importance: 0.00148149650

445 importance: 0.0004104965176351024. feature name: AMZN.US
446 importance: 0.00040877101153870396. feature name: AdjCloseSMA50_above_AdjCloseSMA10_-18
447 importance: 0.0004079199741459737. feature name: AdjClose_above_AdjCloseSMA10_-17
448 importance: 0.00039000434946888433. feature name: AdjClose_crossed_AdjCloseSMA10_-1
449 importance: 0.000385576642942067. feature name: AdjClose_crossed_AdjCloseSMA10_-14
450 importance: 0.00035794029382421973. feature name: AdjClose_crossed_AdjCloseSMA10_-11
451 importance: 0.00032998120705354203. feature name: AdjClose_crossed_AdjCloseSMA10_-13
452 importance: 0.0003168115115433299. feature name: AdjClose_above_AdjCloseSMA50_-8
453 importance: 0.00030097377153362925. feature name: AdjClose_above_AdjCloseSMA200_-19
454 importance: 0.00029167045926850035. feature name: AdjClose_above_AdjCloseSMA200_-12
455 importance: 0.0002826613280455868. feature name: ABBV.US
456 importance: 0.0002798968750273846. feature name: AdjCloseSMA50_crossed_AdjCloseSMA

539 importance: 6.168478074295416e-05. feature name: AdjClose_crossed_AdjCloseSMA50_-3
540 importance: 6.161502020662375e-05. feature name: AdjClose_above_AdjCloseSMA10_-1
541 importance: 5.920334526621887e-05. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-13
542 importance: 5.77406694234877e-05. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-3
543 importance: 5.755880789771278e-05. feature name: AdjClose_crossed_AdjCloseSMA200_-3
544 importance: 5.499111373665495e-05. feature name: AdjCloseSMA50_above_AdjCloseSMA10_-7
545 importance: 5.412165718680456e-05. feature name: AdjClose_crossed_AdjCloseSMA10_-7
546 importance: 5.412165718680456e-05. feature name: AdjClose_above_AdjCloseSMA10_-10
547 importance: 5.3833990466578164e-05. feature name: AdjClose_above_AdjCloseSMA50_-19
548 importance: 5.216548338655008e-05. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-2
549 importance: 4.940774945731664e-05. feature name: AdjClose_above_AdjCloseSMA10_-16
550 importance: 4.9407749457316

652 importance: 0.0. feature name: RSI_above_80_-13
653 importance: 0.0. feature name: RSI_above_80_-14
654 importance: 0.0. feature name: RSI_above_80_-17
655 importance: 0.0. feature name: RSI_above_80_-18
656 importance: 0.0. feature name: RSI_above_80_-19
657 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-11
658 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-10
659 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-6
660 importance: 0.0. feature name: RSI_below_20_-3
661 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-5
662 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-1
663 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_0
664 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-19
665 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-18
666 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-15
667 importance: 0.0. f

At the very bottom of the list are the RSI_above_80 and RSI_below_20 indicators. RSI is an indicator of extremes so having these features not play an important role means that the model did not notice many extremes and having predictive power.
The top section contains many of the SMA50 to SMA200 ratios. The SMA 50 and 200 crossovers or distances are classic momentum indicators (SMA 50 over SMA 200 = uptrend, below = downtrend).
Since the time period in the training data is that of a general market uptrend, it sounds intuitively reasonable that momentum indicators are important.

## 3) Improve RandomForest model with GridSearch

Given the imbalance of our data and the desire to avoid False Positives, we will use a modified F1 score that places stronger emphasis on precision than on recall (they have the same weight in the F1 score). We will skew it towards precision by setting beta to 0.5 and towards recall bysetting beta to 2.0. The combinations were used in separate runs of GridSearchCV and results were saved.

In [15]:
# place higher focus on precision (ie getting most TP and minimize FP) than on recall (ie minimize FN)
# since placing a trade that was based on a FP will be costly and hurts more than missing a trade due to a FN
fhalf_scorer = make_scorer(fbeta_score, beta=0.5) # low beta favors precision over recall
ftwo_scorer = make_scorer(fbeta_score, beta=2) # high beta favors recall over precision

In [16]:
from dask.distributed import Client, LocalCluster, progress
cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:49609 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.26 GB


For a GridSearch, we will define a wide range of parameters to be used in the pipeline. The new fhalf_scorer will be used for optimization.

In [17]:

parameters = {
    'randomForest__min_samples_leaf': [2, 5, 10],
    'randomForest__n_estimators' : [10, 20, 50, 100],
    'randomForest__max_features': [5, 'sqrt', 'log2'], # log2 of 690 = 9, sqrt of 690 = 20
    'randomForest__max_depth' : [4, 5, 6, 7, 8],
    'randomForest__criterion' :['gini', 'entropy']
}
'''
parameters = {
    'randomForest__min_samples_leaf': [2, 5, 10],
    'randomForest__n_estimators' : [10, 20, 50, 100]
    }
'''
my_cv = TimeSeriesSplit(n_splits=3)
cv = dcv.GridSearchCV(pipeline, param_grid=parameters, cv=my_cv, scoring='f1', n_jobs=-1) # uses fhalf_scorer

#with joblib.parallel_backend('dask'):
cv.fit(X_train, y_train)

GridSearchCV(cache_cv=True,
       cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardScaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       iid=True, n_jobs=-1,
       param_grid={'randomForest__n_estimators': [10, 20, 50, 100], 'randomForest__max_depth': [4, 5, 6, 7, 8], 'randomForest__min_samples_leaf': [2, 5, 10], 'randomForest__max_features': [5, 'sqrt', 'log2'], 'randomForest__criterion': ['gini', 'entropy']},
       refit=True, return_train_score='warn', scheduler=None, scoring='f1')

In [37]:
print(cv.best_params_)
pickle.dump(cv.best_params_, open('../models/GridSearch_vbetaone.pkl', 'wb'))

{'randomForest__max_features': 'sqrt', 'randomForest__n_estimators': 10, 'randomForest__max_depth': 8, 'randomForest__criterion': 'entropy', 'randomForest__min_samples_leaf': 10}


In [34]:
print_feature_importances(cv.best_estimator_.named_steps['randomForest'])

Feature ranking:
1 importance: 0.019819236920683308. feature name: week
2 importance: 0.013932401202644324. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-16
3 importance: 0.012933832650140514. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-4
4 importance: 0.012711420987270555. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-6
5 importance: 0.012474221302207652. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-17
6 importance: 0.012403112877513944. feature name: AdjCloseSMA50_-19
7 importance: 0.012301925392493867. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-14
8 importance: 0.011554757394487972. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-16
9 importance: 0.010534778033378853. feature name: AdjCloseSMA50_-16
10 importance: 0.010152821412917862. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-5
11 importance: 0.009186356321784496. feature name: AdjCloseSMA200_to_AdjClose_ratio_-8
12 importance: 0.00884593884802256. feature name: AdjCloseSMA50_to

216 importance: 0.001697076331188201. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-7
217 importance: 0.0016928093648064927. feature name: AdjOpen_-13
218 importance: 0.0016735610926028397. feature name: AdjLow_chg_-12
219 importance: 0.0016650402347949493. feature name: AdjCloseSMA50_to_AdjClose_ratio_-10
220 importance: 0.0016405695238459988. feature name: AdjOpen_chg_-4
221 importance: 0.0016202008903382734. feature name: AdjClose_-1
222 importance: 0.0016195672333592162. feature name: AdjCloseSMA10_to_AdjClose_ratio_-9
223 importance: 0.0016078791529155587. feature name: AdjCloseSMA200_-1
224 importance: 0.0016039475038763153. feature name: AdjVolume_chg_-14
225 importance: 0.0016003383204128481. feature name: AdjCloseSMA200_chg_-5
226 importance: 0.0015997046358071283. feature name: AdjCloseSMA200_-18
227 importance: 0.0015905631215865682. feature name: NFLX.US
228 importance: 0.0015830622318671835. feature name: AdjCloseSMA10_-8
229 importance: 0.0015800591205098083. featu

456 importance: 0.0. feature name: AdjHigh_-17
457 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-14
458 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-15
459 importance: 0.0. feature name: AdjOpen_-2
460 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-16
461 importance: 0.0. feature name: AdjOpen_0
462 importance: 0.0. feature name: AdjCloseSMA50_chg_-6
463 importance: 0.0. feature name: AdjHigh_-18
464 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-17
465 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-12
466 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-18
467 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-19
468 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_0
469 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-1
470 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-2
471 impor

In [35]:
y_pred_GridSearch = cv.predict(X_test)

# save for backtesting in separate notebook
pickle.dump(y_pred_GridSearch, open('../data/model_predictions/y_pred_GridSearch_vbetaone.pkl', 'wb'))

C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [36]:
print(classification_report(y_test, y_pred_GridSearch))

              precision    recall  f1-score   support

       False       0.64      0.86      0.73      1153
        True       0.19      0.06      0.09       603

   micro avg       0.59      0.59      0.59      1756
   macro avg       0.41      0.46      0.41      1756
weighted avg       0.48      0.59      0.51      1756



In [37]:
accuracy_score(y_test, y_pred_GridSearch)

0.5876993166287016

In [38]:
confusion_matrix(y_test, y_pred_GridSearch, labels=[False, True])

array([[995, 158],
       [566,  37]], dtype=int64)

## next: resume in backtesting notebook to understand and measure true performance of the model predictions